<h1 style='text-align:center;color:#ff7392'><strong>Super-Resolution Generative Adversarial Network</strong></h1>

In [1]:
import torch
import config
from torch import nn
from torch import optim
from utils import load_checkpoint, save_checkpoint, plot_examples
from loss import VGGLoss
from torch.utils.data import DataLoader
from model import Generator, Discriminator
from tqdm import tqdm
from dataset import ImageFolder
from train import train_fn

<a id="section_n" name="section_n"></a>

## Table of Contents
* [1. Introduction](#chapter2)
    * [Test Models Work](#section_2_1)
    * [Hyperparameters](#section_2_2)
    * [Load Data](#section_2_3)
    * [Initialize Models](#section_2_4)
    * [Setup Writer Boards and Prepare Training](#section_2_5)
    * [Explaining The Mathematics](#section_2_6)
    * [Train The Network](#section_2_7)

<a id="chapter2" name="chapter2">Chapter 2</a>

<h2 style='text-align:center;color:#ff7392'><strong>1. Introduction</strong></h2>



<a id="section_2_1" name="section_2_1">Section 2.1 </a>

<h4 style="color:#ff7392"><strong>Test Models Work</strong></h4>

In [2]:
import model
model.test()

torch.Size([5, 3, 96, 96])
torch.Size([5, 1])


In [3]:
dataset_load = ImageFolder(dir_name="train")
loader = DataLoader(dataset=dataset_load,batch_size=config.BATCH_SIZE,shuffle=True,pin_memory=True,num_workers=config.NUM_WORKERS)

In [4]:
l,h = iter(loader).next()[0],iter(loader).next()[1]

In [5]:
l.shape,h.shape

(torch.Size([16, 3, 24, 24]), torch.Size([16, 3, 96, 96]))

In [6]:
torch.backends.cudnn.benchmark = True

In [7]:
gen = Generator(in_channels=3).to(config.DEVICE)
disc =Discriminator(in_channels=3).to(config.DEVICE)

In [8]:
opt_gen = optim.Adam(gen.parameters(),lr=config.LEARNING_RATE,betas=(0.9,0.999))
opt_disc = optim.Adam(disc.parameters(),lr=config.LEARNING_RATE,betas=(0.9,0.999))

In [9]:
mse = nn.MSELoss()
bce = nn.BCEWithLogitsLoss()
vgg_loss = VGGLoss()

In [10]:
if config.LOAD_MODEL:
    utils.load_checkpoint(
    config.CHECKPOINT_GEN,
    gen,
    opt_gen,
    config.LEARNING_RATE,
    )
    utils.load_checkpoint(
    config.CHECKPOINT_DISC, disc, opt_disc, config.LEARNING_RATE,
    )

In [11]:
# import os
# from PIL import Image
# import numpy as np

In [12]:
# files = os.listdir("test_images/")[:4]
# for file in files:
#     image = Image.open("test_images/"+file)
#     print(file)
#     print(np.array(image).shape)

In [13]:
for epoch in range(config.NUM_EPOCHS):
    train_fn(loader, disc, gen, opt_gen, opt_disc, mse, bce, vgg_loss)

    if config.SAVE_MODEL:
        save_checkpoint(gen, opt_gen, filename=config.CHECKPOINT_GEN)
        save_checkpoint(disc, opt_disc, filename=config.CHECKPOINT_DISC)


  0%|                                                                                           | 0/50 [00:10<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 676.00 MiB (GPU 0; 6.00 GiB total capacity; 3.13 GiB already allocated; 215.38 MiB free; 3.93 GiB reserved in total by PyTorch)

[back to top](#section_n)